<a href="https://colab.research.google.com/github/albanda/CE888/blob/master/lab5-recommender/rec_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid Models for Recommendation Systems

Load Pandas, we are going to need it for manipulating data

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

Now load the data

In [2]:
user_ratings_df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/user_ratings.csv")
user_features_df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/user_features.csv")
item_features_df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/item_features.csv")

In [3]:
user_features_df["key"] = 0
user_features_df["user_id"] = range(user_features_df.shape[0])
item_features_df["key"] = 0
item_features_df["item_id"] = range(item_features_df.shape[0])

merged_df = pd.merge(user_features_df, item_features_df, left_index=True, on="key")
merged_df.drop(labels='key', axis=1, inplace=True)  # drop the "key" column


In [4]:
merged_df[["item_id", "user_id"]]

,item_id,user_id
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [5]:
merged_df["rating"] = map(lambda ids: user_ratings_df.values[ids[1]][ids[2]], 
                          merged_df[["user_id", "item_id"]].itertuples())

train = merged_df.dropna()
test = merged_df[merged_df.isnull().any(axis=1)]


In [6]:
n_latent_factors = 2

user_ratings = user_ratings_df.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

user_features = user_features_df.values
item_features = item_features_df.values

user_features = np.concatenate([np.ones(shape=(user_features.shape[0],1)), user_features], axis=1)
item_features = np.concatenate([np.ones(shape=(item_features.shape[0],1)), item_features], axis=1)

user_features_weights = np.random.random((user_ratings.shape[0], user_features.shape[1]))
item_features_weights = np.random.random((user_ratings.shape[1], item_features.shape[1]))

print(user_features)

[[1. 1. 0. 0. 0.]
 [1. 0. 1. 0. 1.]
 [1. 0. 0. 0. 2.]
 [1. 1. 0. 0. 3.]
 [1. 0. 1. 0. 4.]
 [1. 0. 0. 0. 5.]
 [1. 0. 0. 0. 6.]
 [1. 1. 0. 0. 7.]
 [1. 0. 1. 0. 8.]
 [1. 1. 0. 0. 9.]]


In [7]:
def predict_rating(user_id, item_id):
    """
    Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    
    user_score = user_features_weights[user_id].dot(user_features[user_id])
    item_score = item_features_weights[item_id].dot(item_features[item_id])
    #print(user_preference.dot(item_preference), user_score, item_score)
    return user_preference.dot(item_preference) + user_score + item_score


def train(user_id, item_id, rating, alpha=0.001, 
          latent_feature_weight_decay=0.1,
          user_weight_decay=0.01, item_weight_decay=0.0001):
    #print(item_id)
    prediction_rating = predict_rating(user_id, item_id)
    err = prediction_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * (latent_item_features[item_id] + latent_feature_weight_decay * latent_user_preferences[user_id])
    latent_item_features[item_id] -= alpha * err * (user_pref_values + latent_feature_weight_decay * latent_item_features[item_id])
    
    user_features_weights[user_id] -=alpha * err * (user_features[user_id] + user_weight_decay * user_features_weights[user_id])
    item_features_weights[item_id] -=alpha * err * (item_features_weights[item_id] + item_weight_decay * item_features_weights[item_id])
    
    return err


def sgd(iterations=30000):
    """ 
    Iterate over all users and all items and train for 
    a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
    mse = (np.array(error) ** 2).mean()          
    print(mse)


In [8]:
for _ in range(10): 
    sgd()  # Note decreasing values with increasing iterations

0.3082337965875413
0.2958460944596951
0.2915812618628391
0.28933668877906177
0.287936598683651
0.28698106340137275
0.2862933764225117
0.2857823581034437
0.28539584310254806
0.28510171864213174


In [9]:
predictions = np.zeros(shape=(latent_user_preferences.shape[0], latent_item_features.shape[0]))
print(user_features_weights)
print(item_features_weights)
for user_id in range(latent_user_preferences.shape[0]):
    for item_id in range(latent_item_features.shape[0]):
        predictions[user_id, item_id] =  predict_rating(user_id, item_id)
  

[[-0.825 -0.799  0.481  0.433  0.047]
 [ 2.829  0.829  3.072  0.523  2.738]
 [ 1.215  0.726  0.88   0.367  1.299]
 [ 0.312  0.35   0.191  0.01   1.309]
 [ 0.157  0.217  0.489  0.037  0.084]
 [-0.648  0.344  0.439  0.279 -3.239]
 [ 0.096  0.276  0.298  0.621  0.063]
 [ 0.706  0.203  0.908  0.246  0.227]
 [ 0.103  0.461  0.938  0.524  0.876]
 [ 0.484 -0.052  0.107  0.467 -0.089]]
[[2.215e-01 2.719e+00 2.655e+00 7.518e-01 5.758e-01]
 [3.388e-03 5.653e-02 1.958e-02 2.670e-02 5.917e-02]
 [4.470e+00 3.617e+00 2.324e+00 1.086e+00 3.844e+00]
 [1.305e+00 9.941e-01 1.392e-01 8.043e-01 6.363e-01]
 [6.647e-02 6.948e-02 5.714e-02 1.233e-01 1.038e-01]]


In [10]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = user_ratings_df.columns

In [11]:
comparison_data


,The Call of Cthulhu,Frankenstein,Dracula,Neuromancer,Space Odyssey
0,"(8.0, 7.919666397490702)","(2.0, 2.3005587367705753)","(nan, -36.29307346217188)","(5.0, 4.78614555097153)","(4.0, 3.996060379755331)"
1,"(3.0, 2.895743878241026)","(2.0, 2.4542455661652336)","(nan, 90.87953148590813)","(7.0, 6.658088122903195)","(7.0, 6.992307103213512)"
2,"(9.0, 8.846390453074278)","(nan, 5.010883533268043)","(7.0, 6.9930909329230015)","(8.0, 8.146050662685397)","(5.0, 5.012305724474845)"
3,"(nan, 9.052699178926979)","(nan, 5.0188837541403135)","(7.0, 6.99983918429686)","(8.0, 8.000037659749951)","(9.0, 9.000064321039556)"
4,"(nan, 4.435520070644719)","(1.0, 0.5060954202371415)","(8.0, 7.989061987430989)","(3.0, 3.468557559865582)","(7.0, 7.0169562188602566)"
5,"(2.0, 2.0145433421891425)","(3.0, 2.9875148990004496)","(5.0, 5.0003504097521105)","(nan, 9.85710732294024)","(nan, -93.50988585303608)"
6,"(4.0, 4.437403750682194)","(2.0, 0.30476739984108075)","(nan, 3.1826535647771745)","(2.0, 3.1928967036257268)","(7.0, 7.0236567998886414)"
7,"(7.0, 6.929175886791984)","(1.0, 2.673421114878234)","(2.0, 2.085069534018391)","(7.0, 5.528777412087625)","(9.0, 8.949915442610449)"
8,"(3.0, 3.063548823835463)","(3.0, 2.786180713249237)","(nan, 89.98966515307897)","(7.0, 7.139668210446685)","(3.0, 3.0024273786628113)"
9,"(4.0, 3.891385412918727)","(nan, 0.03217663254279633)","(5.0, 4.986671936322782)","(3.0, 3.0950318860027055)","(3.0, 3.008005703115507)"
